In [1]:
%pip install opencv-python==4.10.0.84

In [2]:
%pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install ultralytics==8.0.196

Note: you may need to restart the kernel to use updated packages.


In [4]:
import cv2
import numpy as np
import ultralytics
from ultralytics import YOLO
ultralytics.checks()
from cv2 import aruco

Ultralytics YOLOv8.0.196  Python-3.12.5 torch-2.3.1+cpu CPU (AMD Ryzen 5 4500U with Radeon Graphics)
Setup complete  (6 CPUs, 7.4 GB RAM, 202.5/237.2 GB disk)


In [5]:
# Cargamos el modelo entrenado para detección de objetos
modelo = YOLO(r"C:\Users\steven\Downloads\ProyectoVision\best (3).pt")

In [10]:
parametros = cv2.aruco.DetectorParameters()
diccionario = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
detector_aruco = cv2.aruco.ArucoDetector(diccionario, parametros)


In [13]:
# Inicializamos la captura de video desde la cámara con índice 1
cap = cv2.VideoCapture(1)

# Verificamos si la cámara se abrió correctamente
if not cap.isOpened():
    print("Error: No se puede abrir la cámara.")
    exit()

# Bucle para leer y procesar cada frame de video
while True:
    ret, frame = cap.read()  # Leemos el frame de la cámara
    frame_blur = cv2.blur(frame, (3, 3))  # Aplicamos un metodo de procesamiento tradicional: desenfoque (blur) al frame

    # Verificamos si se pudo leer el frame correctamente
    if not ret:
        print("Error: No se puede leer el frame.")
        break

    # Detectamos los marcadores ArUco en el frame desenfocado
    esquinas, ids, _ = detector_aruco.detectMarkers(frame_blur)

    # Si se detectan marcadores ArUco
    if len(esquinas) > 0:
        # Convertimos las esquinas a enteros para dibujar correctamente
        esquinas_ent = [np.int0(esq) for esq in esquinas]
        # Dibujamos polígonos alrededor de los marcadores detectados
        cv2.polylines(frame_blur, esquinas_ent, True, (0, 0, 255), 5) #Dibujamos un poligono de 5 pixeles sobre la imagen desenfocada

        # Calculamos el perímetro del primer marcador ArUco detectado
        perimetro_aruco = cv2.arcLength(esquinas_ent[0], True)

        # Calculamos la proporción en centímetros, para este caso cada del aruco lado mide 9,2
        proporcion_cm = perimetro_aruco / 36.8

    # Detectamos los cascos y gafas en el frame desenfocado utilizando el modelo de YOLOv8 entrenado en Google Colab 
    resultados = modelo(frame_blur, imgsz=640, conf=0.7)

    # Iteramos sobre cada resultado de la detección de objetos
    for resultado in resultados:
        # Obtenemos las coordenadas de la caja delimitadora para cada objeto detectado
        for box in resultado.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Convertimos las coordenadas a enteros
            box_points = np.array([(x1, y1), (x2, y1), (x2, y2), (x1, y2)], dtype=np.int0)

            # Dibujamos el contorno de la caja delimitadora en el frame
            cv2.polylines(frame_blur, [box_points], True, (0, 255, 0), 2)

            # Calculamos el rectángulo de área mínima que encierra la caja delimitadora
            rectangulo = cv2.minAreaRect(box_points)
            (x, y), (an, al), angulo = rectangulo  # Obtenemos la posición, dimensiones y ángulo del rectángulo

            # Calculamos el ancho y alto del objeto detectado en centímetros usando la proporción calculada
            ancho = an / proporcion_cm
            alto = al / proporcion_cm

            # Escribimos el tamaño (alto y ancho) del objeto detectado en la imagen
            cv2.putText(frame_blur, "Alto: {} cm".format(round(ancho, 1)), (int(x), int(y-15)), cv2.LINE_AA, 0.8, (150, 0, 255), 2)
            cv2.putText(frame_blur, "Ancho: {} cm".format(round(alto, 1)), (int(x), int(y+15)), cv2.LINE_AA, 0.8, (75, 0, 75), 2)

    # Mostramos el frame con las detecciones y anotaciones en una ventana emergente
    cv2.imshow('Deteccion', frame_blur)

    # Definimos un tiempo de espera cuando la tecla "ESC" es presionada durante 1 milisegundo
    t = cv2.waitKey(1)
    if t == 27:  # Debemos presionar "ESC" para salir
        break

# Liberamos la cámara y cerramos las ventanas de OpenCV
cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 225.7ms
Speed: 7.5ms preprocess, 225.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 156.4ms
Speed: 3.5ms preprocess, 156.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 126.8ms
Speed: 9.5ms preprocess, 126.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 111.2ms
Speed: 8.8ms preprocess, 111.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 98.1ms
Speed: 2.2ms preprocess, 98.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 99.8ms
Speed: 0.0ms preprocess, 99.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 88.8ms
Speed: 8.1ms preprocess, 88.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 93.8ms
Speed: 2.1ms preprocess, 